In [1]:
# Import packages
import numpy as np
import cvxpy as cp
import mosek
import time
import math
from sklearn.model_selection import train_test_split

import phi_divergence as phi
import robust_sampling as rs
import dataio
import util

The problem we examine is as follows:

Consider a company that sells $n$ different products, which it is able to produce with the use of $m$ different machines. 

The goal is to determine an optimal production plan, which specifies the amount of time $y_{jk}$ that each machine $j=1,\dots,m$ will be used for producing product $k = 1,\dotsc,n$. An optimal plan is one that maximizes the total profit of the company subject to availability constraints.

Each machine $j$ may only be used for a limited amount of time $a_j$ and incurs operating costs $c_{jk}$ per unit of product $k$ that is produced. Each unit of product $k$ can be sold at a price of $u_k$ and leftover units incur inventory holding costs of $\tilde{c}_{k}$. For this problem there are two uncertain parameters, the demand $d_k$ for each product $k$ and the quantity $p_{jk}$ of product $k$ that is produced per time unit by machine $j$. 

Putting all this together, we arrive at the following mathematical formulation:
\begin{align}
    \label{mathform:weighted_dist_1:obj}
    \min_{\mathbf{y}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) \\
    \text{s.t.}&~\sum_{k=1}^{n} y_{jk} \leq a_j, && j=1.\dotsc,m \label{mathform:weighted_dist_1:limit_time}\\
    &~y_{jk} \geq 0, && j=1.\dotsc,m, k=1.\dotsc,n, \label{mathform:weighted_dist_1:xVarNonNeg}
\end{align}


where $[ a ]_{+}$ is equivalent to $\max\{a,0\}$. The $\max$ and $\min$ functions make this a nonlinear optimization problem. As such, it is quite a tricky problem to deal with using conventional Robust Optimization techniques. 

Note that this problem can also be easily converted into the notation used earlier for by moving the uncertainty into the constraints and introducing supplementary variable $\theta$.
Let $\mathbf{x} = (\mathbf{y}, \theta)$ and $\mathscr{X} = \{ \mathbf{x} : \sum_{k=1}^{n} y_{jk} \leq a_j, j=1.\dotsc,m, \mathbf{y} \geq 0 \}$. We can rewrite the problem as:
\begin{align}
    \label{mathform:weighted_dist_2:obj}
    \min_{\mathbf{x} \in \mathscr{X}}&~\sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta  \\
    \text{s.t.}&~\sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta \leq 0.
\end{align}

By defining $g(\mathbf{x}) = \sum_{j=1}^{m} \sum_{k=1}^{n} c_{jk} y_{jk} + \theta$ and $f(\mathbf{x}, \mathbf{z}) = \sum_{k=1}^{n} \tilde{c}_{k} \left[ \sum_{j=1}^{m} p_{jk} y_{jk} - d_k \right]_{+} - \sum_{k=1}^{n} u_k \min \left( \sum_{j=1}^{m} p_{jk} y_{jk}, d_k \right) - \theta$ we obtain the familiar notation used throughout this paper. 

In [16]:
# Problem specific functions:
def generate_data(random_seed, N, **kwargs):
    # specify dimensions and make dist parameters random... 
    return data 

def generate_data_care2014(random_seed, N, **kwargs):
    np.random.seed(random_seed)
    m = 5
    n = 10
    
    # generate demand vector param
    d_nom = (25, 38, 18, 39, 60, 35, 41, 22, 74, 30)
    d = np.random.default_rng().dirichlet(d_nom, N) * 382
    
    # generate production efficiency param
    p_nom = np.array([[5.0, 7.6, 3.6, 7.8, 12.0, 7.0, 8.2, 4.4, 14.8, 6.0],
                      [3.8, 5.8, 2.8, 6.0, 9.2, 5.4, 6.3, 3.4, 11.4, 4.6],
                      [2.3, 3.5, 1.6, 3.5, 5.5, 3.2, 3.7, 2.0, 6.7, 2.7],
                      [2.6, 4.0, 1.9, 4.1, 6.3, 3.7, 4.3, 2.3, 7.8, 3.2],
                      [2.4, 3.6, 1.7, 3.7, 5.7, 3.3, 3.9, 2.1, 7.0, 2.9]])
    p = np.random.random_sample(size = (N,m,n)) * (p_nom*1.05 - p_nom*0.95) + p_nom*0.95
    
    data = [d, p] 
    return data

def solve_P_SCP(S, **kwargs):
    # get fixed parameter values
    C = kwargs['C']
    A = kwargs['A']
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    
    # get uncertain parameter scenarios
    d = S[0]
    p = S[1]
    
    # get dimensions of problem
    m = S[1][0].shape[0]
    n = S[1][0].shape[1]
    num_scen = len(S[0])
    
    # create variables
    theta = cp.Variable(1)
    y = cp.Variable((m, n), nonneg = True)
    
    # set up problem
    constraints = []
    for s in range(num_scen):
        unc_inv_cost_s = sum(C_tilde[k] * cp.maximum(sum(p[s][j][k]*y[j][k] - d[s][k] for j in range(m)), 0)
                             for k in range(n))
        unc_rev_s = sum(U[k] * cp.minimum(sum(p[s][j][k]*y[j][k] for j in range(m)),d[s][k]) 
                        for k in range(n))
        constraints.append(unc_inv_cost_s - unc_rev_s - theta <= 0)
    
    constraints.append(cp.sum(y, axis=1) <= A)
    fixed_costs = cp.sum(cp.multiply(C, y))
    obj = cp.Minimize(fixed_costs + theta)
    prob = cp.Problem(obj,constraints)
    
    # solve problem
    time_limit = kwargs.get('time_limit', 5*60)    
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    x_value = [theta.value, y.value] # Combine y and theta into 1 single solution vector
    return (x_value, prob.value)

def unc_func(scenario, x, **kwargs):
    # extract values
    C_tilde = kwargs['C_tilde']
    U = kwargs['U']
    d = scenario[0]
    p = scenario[1]
    m = p.shape[0]
    n = p.shape[1]
    theta = x[0]
    y = x[1]
    
    # compute function value:
    inv_cost = sum(C_tilde[k] * np.maximum(sum(p[j][k]*y[j][k] - d[k] for j in range(m)), 0)
                       for k in range(n))
    rev = sum(U[k] * np.minimum(sum(p[j][k]*y[j][k] for j in range(m)),d[k]) 
                  for k in range(n))
    
    return inv_cost - rev - theta

def compute_prob_add(lhs_constr):
    method = 'deterministic_w_1%'
    if method == 'deterministic':
        if lhs_constr <= 0:
            return 0
        else:
            return 1
    elif method == 'deterministic_w_1%':
        if lhs_constr <= 0:
            return 0.01
        else:
            return 0.99
    elif method == 'sigmoid':
        return util.compute_prob_add_sigmoid(lhs_constr)
    else:
        print('Error: do not recognize method in "compute_prob_add" function')
        return 1
    
def stopping_cond(stop_info, **kwargs):
    if (kwargs.get('elapsed_time',0) >= stop_info.get('max_elapsed_time', 10e12) 
        or kwargs.get('num_solutions',0) >= stop_info.get('max_num_solutions', 10e12)
        or kwargs.get('num_iterations',0) >= stop_info.get('max_num_iterations', 10e12)):
        return True
    else:
        return False

def analytic_eval(x, problem_info):
    return ...

In [3]:
# fixed parameter values from Care (2014)
C = np.array([[1.8, 2.2, 1.5, 2.2, 2.6, 2.1, 2.2, 1.7, 2.8, 1.9],
              [1.6, 1.9, 1.3, 1.9, 2.3, 1.9, 2.0, 1.5, 2.5, 1.7],
              [1.2, 1.5, 1.0, 1.5, 1.9, 1.4, 1.6, 1.1, 2.0, 1.3],
              [1.3, 1.6, 1.1, 1.6, 2.0, 1.5, 1.7, 1.2, 2.2, 1.4],
              [1.2, 1.5, 1.0, 1.6, 1.9, 1.5, 1.6, 1.1, 2.1, 1.3]])

A = np.array([10, 13, 22, 19, 21])
C_tilde = np.array([1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3])
U = np.array([1.5, 1.8, 1.2, 1.9, 2.2, 1.8, 1.9, 1.4, 2.4, 1.6])

In [7]:
param_dict = {'C':C, 'A':A, 'C_tilde': C_tilde, 'U': U}

In [6]:
random_seed = 0
N = 10 #10580
data = generate_data_care2014(random_seed, N)

In [20]:
for N in [1, 10, 100, 1000]:
    data = generate_data_care2014(random_seed, N)
    print(solve_P_SCP(data, **param_dict)[1])

-593.8918782115552
-560.1051275627121
-535.2741449049572
-523.6679785942181


In [ ]:
# Care
random_seed = 0
N = 10580
data = generate_data_care2014(random_seed, N)

%timeit
x, obj = solve_P_SCP(data, **param_dict)